In [1]:
import joblib
import pandas as pd
import numpy as np
import string
import unicodedata
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from joblib import dump, load
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
file = '/content/drive/MyDrive/OFF_Drive/raw_data/ocr_labeled_spellcheck.csv'
data = pd.read_csv(file)


In [ ]:
#data = pd.read_csv('ocr_labeled_spellcheck.csv')

In [4]:
data = data.copy()

In [5]:
data.shape

(434896, 6)

In [9]:
data.head()

,barcode,clean_text,fr_text,source,pnns_groups_1,pnns_groups_2
0,3199660476748,ne eleve abattu en bretagne les eleveurs de br...,NE\nELEVE\nABATTU\nEN BRETAGNE\nLES ÉLEVEURS\n...,/319/966/047/6748/1.json,fish meat eggs,meat
1,3199660219192,mer chant local decoupes de poulet conditionne...,Ker\nchant\n100% LOCAL\nDecoupes de\nPOULET\nC...,/319/966/021/9192/1.json,fish meat eggs,meat
2,3199660219192,cuis dej plat sat merchant ex origine france v...,1CUIS.DEJ. PLT SAT\nKERCHANT FX 1\nORIGINE Fra...,/319/966/021/9192/2.json,fish meat eggs,meat
3,3199660747848,mer chant local decoupes de poulet cooduogne p...,Ker\nChant\n100% LOGAL\nDécoupes de\nPOULET\nC...,/319/966/074/7848/1.json,fish meat eggs,meat
4,3196203800091,preparation deshydratee aromatisee et coloree ...,"PREPARATION DÉSHYDRATÉE, AROMATISÉE ET COLOREE...",/319/620/380/0091/1.json,sugary snacks,sweets


In [8]:
#install Spacy
!pip3 install spacy

In [9]:
# French library
!python3 -m spacy download fr_core_news_md

     |████████████████████████████████| 88.6MB 1.6MB/s 
  Created wheel for fr-core-news-md: filename=fr_core_news_md-2.2.5-cp37-none-any.whl size=90338490 sha256=d559bfcd7675df63492e190f6eabe824f3a13501656368cabdbcf411c462e801
  Stored in directory: /tmp/pip-ephem-wheel-cache-bd18lixq/wheels/c6/18/b6/f628642acc7872a53cf81269dd1c394d96da69564ccfac5425
Successfully built fr-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')


In [6]:
# Sur Google Colab you must restart execution after downloading fr_core_news_md
import spacy
import fr_core_news_md
nlp = spacy.load('fr_core_news_md')


In [15]:
# Stopwords & Lemmatize
def stopwords(text): 
    doc = nlp(text)  
    noStopWords = [t for t in doc if not t.is_stop]
    for token in noStopWords:
        lema = [token.lemma_ for token in noStopWords]
        new_text = ', '.join(lema)
        return new_text


In [16]:
#test the function
stopwords(data['clean_text'][0])

'eleve, abattre, bretagne, eleveur, bretagne, decoupe, poulet, noir, fermier, labo, caracteristique, certifiees, fermier, eleve, air, duree, d, elevage, jour, minimum, cereales, alimente, vegetau, mineral, vitamine, do, matin, voir, etiquette, poids, prix, conserver, oc, c, consommer, cuit, coeur, date, limite, co, heu, certifier, immeuble, millepertuis, lande, d, ap, produit, frais, classe, reclamatin, s, adresser, fermier, d, argot, bp, ploufragan, decoupe, conditionn, lac, bretagne, bp, quentin, homologation, n, abattoir, agre, n, fr, être, plat, fer, lr, sat, el, bret, origine, france, yo, volai, prixtrg, poids, net, prix, payer, kilogramme, lot, r, consommer, r, conserver, c, c, expedie, loc, bretagne, lanfains, e, toit, acte, wow, consignesdetrfr'

In [17]:
# Apply the function on the clean_text column 
data['stop_worded'] = data['clean_text'].apply(stopwords)

In [23]:
#Have a look 
data.shape

(434896, 7)

In [34]:
#To be sure that the dataset contains only strings after this treatment (if not it will return an error when training the model)
data['stop_worded']= data['stop_worded'].apply(lambda x: str(x))


In [ ]:
# I save the preprocessed csv for Open Food Facts if thay want to train another model on this dataset. Applying Spacy took me about 3h30. 
data.to_csv('/content/drive/My Drive/OFF_Drive/raw_data/data_preprocessed.csv', index = False)

In [18]:
# Give a number to each catégories (for classification report)
category_codes_pnns_2 = {n: i for i, n in enumerate(sorted(data['pnns_groups_2'].unique()))}


In [35]:
# Holdout
X = data[['stop_worded']]

y = data['pnns_groups_2']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1)

In [36]:
# Pipeline
ridge_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(ngram_range=(2, 2))),
                ('Ridge', RidgeClassifier()),
            ])
  
# train the model using X_dtm & y
ridge_pipeline.fit(X_train.squeeze(), y_train)
# Squeeze is to get the correct shape for X (if none I have a 1 column after my rows)
# compute the testing accuracy
prediction = ridge_pipeline.predict(X_test.squeeze())
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))
print(classification_report(y_test, prediction, target_names=category_codes_pnns_2))

Test accuracy is 0.8170446619503483
                                  precision    recall  f1-score   support

                      appetizers       0.85      0.76      0.80      3575
artificially sweetened beverages       0.81      0.57      0.66       615
              biscuits and cakes       0.82      0.89      0.86     11401
                           bread       0.90      0.79      0.84      3311
               breakfast cereals       0.90      0.87      0.88      2149
                         cereals       0.82      0.79      0.81      5257
                          cheese       0.88      0.85      0.87      5937
              chocolate products       0.88      0.88      0.88      2554
                  dairy desserts       0.85      0.73      0.79      2943
            dressings and sauces       0.85      0.78      0.81      5200
                    dried fruits       0.86      0.76      0.81      1451
                            eggs       0.95      0.87      0.91       768
 

In [37]:
from sklearn.model_selection import cross_validate

# 5-Fold Cross validate model
cv_results = cross_validate(ridge_pipeline, X_train.squeeze(), y_train, cv=5)

# Scores
cv_results['test_score']

# Mean of scores
cv_score = cv_results['test_score'].mean()

cv_score

0.8086930525355955

In [38]:
# all the scores
cv_results['test_score']



array([0.80938147, 0.80810038, 0.80776874, 0.80931264, 0.80890203])

In [39]:
model = ridge_pipeline.fit(X_train.squeeze(), y_train)

In [40]:
# Export model, joblib format
joblib.dump(model, '/content/drive/My Drive/OFF_Drive/raw_data/models/ridge_model_pnns_2.joblib') 
# It's a Google Colab path, to be adapted to your usage

['/content/drive/My Drive/OFF_Drive/raw_data/models/ridge_model_pnns_2.joblib']

In [ ]:
#to load jobil model:
#loaded_model = joblib.load('/content/drive/My Drive/OFF_Drive/raw_data/models/ridge_only_pnns_2.joblib')
#result = loaded_model.score(X_test.squeeze(), y_test)

#To predict:
#loaded_model.predict([['stop_worded'][0]])

In [42]:
# Export model, pikle format
pkl_filename = '/content/drive/My Drive/OFF_Drive/raw_data/models/ridge_model_pnns_2.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

#to predict:
#pickle_model.predict(Xtest)